In [70]:
import random
random.seed = 16
import pandas as pd
from gensim.models.word2vec import Word2Vec
import jieba
from collections import Counter
import numpy as np
import os
import pickle
print("import ended...")

import ended...


In [71]:
print("read csv started...")
data = pd.read_csv("ai_challenger_sentiment_analysis_trainingset_20180816/sentiment_analysis_trainingset.csv")
data_validation = pd.read_csv("ai_challenger_sentiment_analysis_validationset_20180816/sentiment_analysis_validationset.csv")
data_test = pd.read_csv("ai_challenger_sentiment_analysis_testa_20180816/sentiment_analysis_testa.csv")
print("training.shape:",data.shape)
print("data_validation.shape:",data_validation.shape)
print("data_test.shape:",data_test.shape)
print("read csv ended...")

read csv started...
training.shape: (105000, 22)
data_validation.shape: (15000, 22)
data_test.shape: (15000, 22)
read csv ended...


In [75]:
#print(data.describe())

In [76]:
print(data.head(n=1))

   id                                            content  \
0   0  "吼吼吼，萌死人的棒棒糖，中了大众点评的霸王餐，太可爱了。一直就好奇这个棒棒糖是怎么个东西，...   

   location_traffic_convenience  location_distance_from_business_district  \
0                            -2                                        -2   

   location_easy_to_find  service_wait_time  service_waiters_attitude  \
0                     -2                 -2                         1   

   service_parking_convenience  service_serving_speed  price_level  \
0                           -2                     -2           -2   

                ...                 environment_decoration  environment_noise  \
0               ...                                     -2                 -2   

   environment_space  environment_cleaness  dish_portion  dish_taste  \
0                 -2                    -2            -2          -2   

   dish_look  dish_recommendation  others_overall_experience  \
0          1                   -2                          1   


In [77]:
 # id,content,location_traffic_convenience,location_distance_from_business_district,location_easy_to_find,service_wait_time,service_waiters_attitude,service_parking_convenience,service_serving_speed,price_level,price_cost_effective,price_discount,environment_decoration,environment_noise,environment_space,environment_cleaness,dish_portion,dish_taste,dish_look,dish_recommendation,others_overall_experience,others_willing_to_consume_again
#  0,"""吼吼吼，萌死人的棒棒糖，中了大众点评的霸王餐，太可爱了。一直就好奇这个棒棒糖是怎么个东西，大众点评给了我这个土老冒一个见识的机会。看介绍棒棒糖是用德国糖做的，不会很甜，中间的照片是糯米的，能食用，真是太高端大气上档次了，还可以买蝴蝶结扎口，送人可以买礼盒。我是先打的卖家电话，加了微信，给卖家传的照片。等了几天，卖家就告诉我可以取货了，去大官屯那取的。虽然连卖家的面都没见到，但是还是谢谢卖家送我这么可爱的东西，太喜欢了，这哪舍得吃啊。""",-2,-2,-2,-2,1,-2,-2,-2,-2,1,-2,-2,-2,-2,-2,-2,1,-2,1,-2

#print(data.head(n=1))
data.shape

(105000, 22)

In [80]:
# print some line to have a look
for index, row in data.iterrows():
    id_=row['id']
    content=row['content']
    label=row[2:]
    print("id_:",id_)
    print("content:",content)
    print("label:",label)
    print("===============================================================")
    if index==5: 
        break

id_: 0
content: "吼吼吼，萌死人的棒棒糖，中了大众点评的霸王餐，太可爱了。一直就好奇这个棒棒糖是怎么个东西，大众点评给了我这个土老冒一个见识的机会。看介绍棒棒糖是用德国糖做的，不会很甜，中间的照片是糯米的，能食用，真是太高端大气上档次了，还可以买蝴蝶结扎口，送人可以买礼盒。我是先打的卖家电话，加了微信，给卖家传的照片。等了几天，卖家就告诉我可以取货了，去大官屯那取的。虽然连卖家的面都没见到，但是还是谢谢卖家送我这么可爱的东西，太喜欢了，这哪舍得吃啊。"
label: location_traffic_convenience                -2
location_distance_from_business_district    -2
location_easy_to_find                       -2
service_wait_time                           -2
service_waiters_attitude                     1
service_parking_convenience                 -2
service_serving_speed                       -2
price_level                                 -2
price_cost_effective                        -2
price_discount                               1
environment_decoration                      -2
environment_noise                           -2
environment_space                           -2
environment_cleaness                        -2
dish_portion                                -2
dish_taste                                  -2
dish

In [81]:
def get_sentiment_analysis_labels(row):
    # 1)location
    location_traffic_convenience = row['location_traffic_convenience']               
    location_distance_from_business_district= row['location_distance_from_business_district'] 
    location_easy_to_find  = row['location_easy_to_find']  
    # 2)service
    service_wait_time  = row['service_wait_time']                         
    service_waiters_attitude = row['service_waiters_attitude']                     
    service_parking_convenience = row['service_parking_convenience']                 
    service_serving_speed  = row['service_serving_speed']  
    # 3)price
    price_level         = row['price_level']                        
    price_cost_effective = row['price_cost_effective']                          
    price_discount  = row['price_discount']     
    # 4)environment
    environment_decoration  = row['environment_decoration']                   
    environment_noise   = row['environment_noise']                        
    environment_space   = row['environment_space']                         
    environment_cleaness     = row['environment_cleaness']  
    # 5)dish
    dish_portion   = row['dish_portion']                              
    dish_taste =row['dish_taste']                                   
    dish_look  = row['dish_look']                                  
    dish_recommendation = row['dish_recommendation']   
    # 6)other
    others_overall_experience  = row['others_overall_experience']                    
    others_willing_to_consume_again   = row['others_willing_to_consume_again']   
    
    label_list=[]
    label_list=[location_traffic_convenience,location_distance_from_business_district,location_easy_to_find, # location
                  service_wait_time,service_waiters_attitude,service_parking_convenience,service_serving_speed, # service
               price_level,price_cost_effective,price_discount, # price
               environment_decoration,environment_noise,environment_space,environment_cleaness, # environment
               dish_portion,dish_taste,dish_look,dish_recommendation, # dish
               others_overall_experience,others_willing_to_consume_again] # other
    label_list=[str(i)+"_"+str(label_list[i]) for i  in range(len(label_list))]
    return label_list 

In [82]:
def convet_to_one_hot(label_list,num_classes=80):
    new_label_list=[0 for i in range(num_classes)]
    for label in label_list:
        new_label_list[label]=1
    return new_label_list

label_list=[0, 4, 8, 12, 19, 20, 24, 28, 32, 39, 43, 47, 51, 55, 59, 62, 64, 68, 75, 76]
label_list_one_hot=convet_to_one_hot(label_list)
print("label_list_one_hot:",label_list_one_hot)

label_list_one_hot: [1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]


In [83]:
 # print first row
 for index, row in data.iterrows():
    id_=row['id']
    content=[x for x in jieba.lcut(row['content']) if x.strip() and x!="\""]
    sentiment_label_list=get_sentiment_analysis_labels(row)
    print("id_:",id_)
    print("content:",content)
    print("sentiment_label_list:",sentiment_label_list,";length:",len(sentiment_label_list))
    if index==0: 
        break

id_: 0
content: ['吼吼', '吼', '，', '萌死', '人', '的', '棒棒糖', '，', '中', '了', '大众', '点评', '的', '霸王餐', '，', '太', '可爱', '了', '。', '一直', '就', '好奇', '这个', '棒棒糖', '是', '怎么', '个', '东西', '，', '大众', '点评', '给', '了', '我', '这个', '土老冒', '一个', '见识', '的', '机会', '。', '看', '介绍', '棒棒糖', '是', '用', '德国', '糖', '做', '的', '，', '不会', '很甜', '，', '中间', '的', '照片', '是', '糯米', '的', '，', '能', '食用', '，', '真是太', '高端', '大气', '上档次', '了', '，', '还', '可以', '买', '蝴蝶结', '扎口', '，', '送人', '可以', '买', '礼盒', '。', '我', '是', '先', '打', '的', '卖家', '电话', '，', '加', '了', '微信', '，', '给', '卖家', '传', '的', '照片', '。', '等', '了', '几天', '，', '卖家', '就', '告诉', '我', '可以', '取货', '了', '，', '去', '大官', '屯', '那取', '的', '。', '虽然', '连', '卖家', '的', '面', '都', '没', '见到', '，', '但是', '还是', '谢谢', '卖家', '送', '我', '这么', '可爱', '的', '东西', '，', '太', '喜欢', '了', '，', '这', '哪', '舍得吃', '啊', '。']
sentiment_label_list: ['0_-2', '1_-2', '2_-2', '3_-2', '4_1', '5_-2', '6_-2', '7_-2', '8_-2', '9_1', '10_-2', '11_-2', '12_-2', '13_-2', '14_-2', '15_-2', '16_1', '17_-2', '18_1', '

In [90]:
data_big=pd.concat([data_traininig_small,data_validation_small,data_test_small])#(data_traininig_small, data_test_small, on='key')
print("data_big:",data_big.shape)

data_big: (135000, 22)


In [92]:
# sample a some percentage of data(set frac to small value when you are in test model) and create vocabulary by get most requency words
vocab_size=70000

print("df.shape:",data.shape)

PAD_ID = 0
UNK_ID=1
CLS_ID=2
MASK_ID=3
_PAD="_PAD"
_UNK="UNK"
_CLS="CLS"
_MASK="MASK"

data_traininig_small=data.sample(frac=1.0)
data_validation_small=data_validation.sample(frac=1.0)
data_test_small=data_test #[0:1000]
print("data_traininig_small.shape:",data_traininig_small.shape)
print("data_validation_small.shape:",data_validation_small.shape)
print("data_test_small.shape:",data_test_small.shape)

num_example,_=data_traininig_small.shape
print("num_example:",num_example)
total_length=0
c_inputs=Counter()
count_index=0
for index, row in data_big.iterrows():
    #id_=row['id']
    input_list=[x for x in jieba.lcut(row['content']) if x.strip() and x!="\""]
    total_length+=len(input_list)
    c_inputs.update(input_list)
    count_index=count_index+1
    if count_index%5000==0: 
        print("count.create vocabulary of words:",count_index)
        
vocab_list=c_inputs.most_common(vocab_size-4)
print("vocab_list:",vocab_list)
vocab_word2index={}
vocab_word2index[_PAD]=PAD_ID
vocab_word2index[_UNK]=UNK_ID
vocab_word2index[_CLS]=CLS_ID
vocab_word2index[_MASK]=MASK_ID

count_index=0
for i,tuplee in enumerate(vocab_list):
    word,freq=tuplee
    vocab_word2index[word]=i+4
print("vocab_word2index:",vocab_word2index)

df.shape: (105000, 22)
data_traininig_small.shape: (105000, 22)
data_validation_small.shape: (15000, 22)
data_test_small.shape: (15000, 22)
num_example: 105000
count.create vocabulary of words: 5000
count.create vocabulary of words: 10000
count.create vocabulary of words: 15000
count.create vocabulary of words: 20000
count.create vocabulary of words: 25000
count.create vocabulary of words: 30000
count.create vocabulary of words: 35000
count.create vocabulary of words: 40000
count.create vocabulary of words: 45000
count.create vocabulary of words: 50000
count.create vocabulary of words: 55000
count.create vocabulary of words: 60000
count.create vocabulary of words: 65000
count.create vocabulary of words: 70000
count.create vocabulary of words: 75000
count.create vocabulary of words: 80000
count.create vocabulary of words: 85000
count.create vocabulary of words: 90000
count.create vocabulary of words: 95000
count.create vocabulary of words: 100000
count.create vocabulary of words: 105000

In [95]:
# compute length distribution of inputs and set max_sequence_length; 
# choose a mininum value of max_sequence_length that let 90% of inputs' s length is less of it.
total_length=0
input_length_list=[50,100,150,200,250,300,350,400,500,600,700,1000]
input_length_dict={x:0 for x in input_length_list }
data_big_part=data_big.sample(frac=0.25)

num_example_for_length,_=data_big_part.shape
print("num_example_for_length:",num_example_for_length)
for index, row in data_big_part.iterrows(): # data_traininig_small
    id_=row['id']
    input_list=[x for x in jieba.lcut(row['content']) if x.strip() and x!="\""]
    length_input=len(input_list)
    fixed_len=1000
    for length in input_length_list:
        if length_input<length:
            fixed_len=length
            break
    input_length_dict[fixed_len]=input_length_dict[fixed_len]+1
    total_length+=length_input
    c_inputs.update(input_list)
    
avg_length=float(total_length) /float(num_example_for_length)
print("avg_length:",avg_length)
print("input_length_dict:",input_length_dict)

input_length_dict_percentage={}
for k,v in input_length_dict.items():
    v=v/num_example_for_length
    input_length_dict_percentage[k]=v
print("input_length_dict_percentage:",input_length_dict_percentage)

# conclusion: most length is between 150-250. average length is 216. if we use max length 350, then we can cover about:  91% of inputs.
# choose a mininum value of max_sequence_length that let 90% of inputs' s length is less of it.
max_sequence_length=0
accumulate_percentage=0
for fixed_length,percentage in input_length_dict_percentage.items():
    accumulate_percentage+=percentage
    if accumulate_percentage>0.9:
        max_sequence_length=fixed_length
        print("max_sequence_length:",max_sequence_length)
        break

num_example_for_length: 33750
avg_length: 218.16085925925927
input_length_dict: {50: 6, 100: 62, 150: 9409, 200: 10898, 250: 5321, 300: 2764, 350: 1914, 400: 1095, 500: 1142, 600: 577, 700: 232, 1000: 330}
input_length_dict_percentage: {50: 0.00017777777777777779, 100: 0.001837037037037037, 150: 0.2787851851851852, 200: 0.3229037037037037, 250: 0.15765925925925925, 300: 0.0818962962962963, 350: 0.05671111111111111, 400: 0.03244444444444444, 500: 0.03383703703703704, 600: 0.017096296296296297, 700: 0.006874074074074074, 1000: 0.009777777777777778}
max_sequence_length: 400


In [96]:
 # create dict of label to index
value_list=[-2,-1,0,1]
num_aspect=20
label2index={}
count_label=0
for i in range(num_aspect):
    for value in value_list:
        label2index[str(i)+"_"+str(value)]=count_label
        count_label+=1
print("label2index:",label2index)


label2index: {'0_-2': 0, '0_-1': 1, '0_0': 2, '0_1': 3, '1_-2': 4, '1_-1': 5, '1_0': 6, '1_1': 7, '2_-2': 8, '2_-1': 9, '2_0': 10, '2_1': 11, '3_-2': 12, '3_-1': 13, '3_0': 14, '3_1': 15, '4_-2': 16, '4_-1': 17, '4_0': 18, '4_1': 19, '5_-2': 20, '5_-1': 21, '5_0': 22, '5_1': 23, '6_-2': 24, '6_-1': 25, '6_0': 26, '6_1': 27, '7_-2': 28, '7_-1': 29, '7_0': 30, '7_1': 31, '8_-2': 32, '8_-1': 33, '8_0': 34, '8_1': 35, '9_-2': 36, '9_-1': 37, '9_0': 38, '9_1': 39, '10_-2': 40, '10_-1': 41, '10_0': 42, '10_1': 43, '11_-2': 44, '11_-1': 45, '11_0': 46, '11_1': 47, '12_-2': 48, '12_-1': 49, '12_0': 50, '12_1': 51, '13_-2': 52, '13_-1': 53, '13_0': 54, '13_1': 55, '14_-2': 56, '14_-1': 57, '14_0': 58, '14_1': 59, '15_-2': 60, '15_-1': 61, '15_0': 62, '15_1': 63, '16_-2': 64, '16_-1': 65, '16_0': 66, '16_1': 67, '17_-2': 68, '17_-1': 69, '17_0': 70, '17_1': 71, '18_-2': 72, '18_-1': 73, '18_0': 74, '18_1': 75, '19_-2': 76, '19_-1': 77, '19_0': 78, '19_1': 79}


In [97]:
 # generate training data(X,y) by using vocabulary of words, padding sequence
def process_data(data,data_type='train_valid'):
    X,Y=[],[]
    print(data.shape)
    count=0
    for index, row in data.iterrows():
            input_list=[x for x in jieba.lcut(row['content']) if x.strip() and x!="\""]
            x=[vocab_word2index.get(x,UNK_ID) for x in input_list]
            X.append(x)
            if data_type=='train_valid':
                sentiment_label_list=get_sentiment_analysis_labels(row)  
                y_dense=[label2index[label] for label in sentiment_label_list]
                y=convet_to_one_hot(y_dense)
                Y.append(y)
                count=count+1
                if count%10000==0:
                    if data_type=='train_valid':
                        print(count,",sentiment_label_list:",sentiment_label_list)
                        print(count,",y_dense:",y_dense) 
                        print(count,",y:",y) 
                    print("===================================================")
            if count%10000==0:
                print(count,",input_list:",input_list)
                print(count,",x:",x)
    return X,Y

print("process_data.started...")
X_train,Y_train=process_data(data_traininig_small)
print("process_data.train.ended.")
X_valid,Y_valid=process_data(data_validation_small)
print("process_data.valid.ended.")
X_test,_=process_data(data_test_small,data_type='test')
print("process_data.ended...")

print("X_train:",np.array(X_train).shape,";Y_train:",np.array(Y_train).shape)
print("X_valid:",np.array(X_valid).shape,";Y_valid:",np.array(Y_valid).shape)
print("X_test:",np.array(X_test).shape)
print("X_train[0]:",X_train[0])

process_data.started...
(105000, 22)
5000 ,input_list: ['建立', '在', '火车站', '附近', '，', '人流', '也', '就', '没得说', '，', '我', '只', '想', '说', '，', '点', '了', '个', '套餐', '，', '只', '给', '一张', '纸', '，', '请问', '给', '你', '吃', '薯条', '和', '汉堡', '一张', '纸够', '吗', '？', '后来', '去', '服务台', '要', '，', '等', '了', '大半天', '才', '给', '那', '的', '，', '这是', '真的', '有', '这么', '忙', '吗', '？', '要', '回家', '了', '，', '然后', '一个', '人来', '的', '，', '想', '去', '个', '洗手间', '，', '背', '的', '包包', '拿到', '服务台', '想', '让', '他们', '给', '看', '一下', '，', '那个', '男', '的', '好像', '还是', '组长', '来', '的', '，', '给', '我', '来', '了', '一句', '，', '人太多', '，', '我们', '看', '不住', '！', '丢', '了', '没法', '负责', '，', '你', '还是', '自己', '拿走', '吧', '，', '我', '也', '是', '醉', '了', '，', '吃', '了', '不下', '30', '家', '麦当劳', '，', '这个', '我', '真的', '觉着', '里', '倒闭', '不远', '了', '，', '而是', '这边', '的', '服务', '只会', '得罪', '更', '多', '的', '客人', '，', '原本', '建立', '在', '火车站', '附近', '就是', '给予', '来往', '的', '行人', '方便', '，', '这', '倒好', '，', '以后', '再也不会', '来', '和', '推荐', '朋友', '来', '这边', '了', '.......

25000 ,input_list: ['霸王餐', '中', '的', '，', '真是', '幸运', '，', '第一次', '听见', '遇见', '澳洲', '鱼', '这个', '名字', '就', '好', '美', '，', '位置', '好找', '，', '三号', '线', '狮子', '坪', '站', '，', '出站', '步行', '几百米', '就', '到', '，', '店', '比较', '大', '，', '环境', '比较', '温馨', '，', '还', '挂', '着', '好多', '可爱', '的', '玻璃', '鱼', '，', '进门', '服务员', '相当', '热情', '，', '引领到', '我们', '那桌', '，', '做', '任何', '一件', '事', '都', '不', '忘', '说', '打扰', '，', '请', '慢用', '等', '词语', '，', '训练有素', '！', '首先', '是', '鱼骨', '熬', '的', '汤', '，', '里面', '还', '能', '闻到', '一点', '鸡汤', '味儿', '，', '比较', '鲜', '，', '佐料', '是', '自助', '的', '，', '可以', '依据', '自己', '口味', '儿调', '，', '鱼片', '切得', '相当', '薄', '，', '好', '刀功', '！', '在', '鱼汤', '里', '涮', '涮', '就', '可以', '吃', '，', '十分', '鲜美', '。', '鱼刺', '身', '也', '很', '美味', '，', '新鲜', '。', '最爱', '的', '是', '凉菜', '里', '的', '莼菜', '，', '爽滑', '可口', '。', '还有', '蛋卷', '鱼滑', '，', '吃', '起来', '十分', '美味', '。', '店里', '还', '送', '了', '薏仁', '紫薯', '饮料', '，', '美味', '暖', '胃', '。', '还有', '油泼', '豆花', '鱼', '，', '特别', '适合', '重庆', '人口', '味儿', '。', '总之', '

45000 ,input_list: ['两节', '期间', '，', '他们', '家', '还', '蛮', '火', '的', '，', '上周', '总算', '订到', '了', '最后', '一个', '包间', '。', '南海', '街', '修路', '，', '不', '过去', '他家', '还', '可以', '走', '便', '道', '。', '包间', '在', '一层', '，', '先', '早早', '下', '了', '菜单', '，', '两', '热菜', '都', '比较', '快', '，', '主食', '略慢', '。', '凉菜', ':', '浑源', '凉粉', '。', '招牌', '，', '确实', '好吃', '，', '豆干', '和', '蚕豆', '搭配着', '很香', '，', '凉粉', '劲', '道', '，', '汤汁', '不辣', '。', '木耳', '拌', '田七', '，', '比较', '清淡', '，', '田七', '有', '清香味', '。', '猪皮', '冻', '，', '据说', '自制', '，', '水晶', '样', '，', '猪皮', '劲', '道', '，', '配醋', '即可', '。', '浇汁', '牛肉', '，', '是', '除了', '凉粉', '外', '特别', '要', '推荐', '的', '凉菜', '，', '肉片', '厚薄', '适度', '，', '不柴', '不', '硬', '，', '浇汁', '看似', '麻辣', '，', '实则', '不辣', '麻香', '，', '口感', '很赞', '！', '热菜', ':', '椒', '香雅鱼', '，', '没', '刺', '，', '我', '感觉', '鱼肉', '像', '龙利鱼', '。', '看上去', '味重', '偏辣', '，', '其实', '不辣', '，', '肉质', '嫩', '，', '有', '老人', '小朋友', '可以', '考虑', '。', '他家', '鱼类', '菜肴', '种类', '不', '多', '。', '火锅', '，', '下图', '是', '小火锅', '，', '但料', '不少

60000 ,input_list: ['首先', '还是', '要', '感谢', '大众', '点评', '能', '给', '我', '一次', '抽中', '100', '元代', '金', '劵', '的', '机会', '，', '同时', '也', '要', '感谢', '上味', '人家', '火锅', '提供', '给', '大家', '这样', '给力', '的', '活动', '。', '其实', '我', '是', '在', '抽到', '代金', '劵', '后', '没', '几天', '就', '去', '店里', '消费', '了', '，', '感觉', '挺', '好', '的', '，', '环境', '很', '幽雅', '，', '干净', '，', '服务态度', '也', '很', '好', '。', '这家', '店', '也', '很', '好找', '，', '基本上', '去', '一品', '天下', '随便', '逛逛', '就', '可以', '找到', '，', '坐下', '来', '第一', '时间', '就', '去', '拿', '了', '哈密瓜', '和', '西瓜', '来', '吃', '，', '水果', '很', '新鲜', '很甜', '，', '点', '的', '鸳鸯', '火锅', '味道', '好吃', '，', '尤其', '烫', '脑花', '感觉', '味道', '很赞', '，', '其它', '菜品', '也', '很', '新鲜', '，', '用', '这个', '代金', '劵', '真的', '感觉', '很', '划算', '呢', '，', '希望', '这种', '活动', '能', '多', '做', '几次', '，', '也', '算是', '给', '吃货', '们', '的', '福利', '吧', '。']
60000 ,x: [360, 27, 49, 464, 163, 103, 106, 67, 16, 234, 982, 470, 8317, 2376, 2431, 5, 442, 4, 1325, 11, 49, 464, 29080, 1005, 182, 672, 67, 185, 219, 2581, 5, 207, 6,

80000 ,input_list: ['季季', '红', '在', '南昌', '开', '了', '很多年', '，', '从', '原来', '的', '脏乱', '便宜', '逐渐', '升级', '变更', '为', '实惠', '环境', '好', '，', '到', '季季', '红', '吃饭', '的', '基本', '都', '是', '学生', '及', '一些', '年轻', '群体', '，', '大家', '都', '冲着', '价格', '实惠', '，', '环境', '不错', '去', '的', '。', '这次', '在', '朋友', '要求', '下去', '了', '师大', '店', '，', '外', '观看', '店', '不错', '，', '上', '台阶', '发现', '店里', '装修', '设置', '已经', '有季', '季红', '想要', '的', '特色', '，', '说明', '季季', '红', '也', '在', '努力', '升级', '并', '改变', '自身', '，', '也', '在', '力求', '将', '火锅店', '打', '造成', '精品', '特色', '火锅店', '。', '点单', '实行', '自助', '点单', '，', '但', '也许', '是', '人气', '旺', '，', '不怕', '没', '客源', '，', '在', '买单', '时', '竟然', '不能', '刷', '信用卡', '，', '这点', '有点', '遗憾', '，', '食物', '端', '来', '，', '服务', '也', '较', '规范', '，', '特意', '点', '了', '川麻', '口味', '锅底', '，', '品尝', '后', '口感', '真的', '很', '一般', '，', '和', '在', '四川', '成都', '吃', '的', '火锅', '简直', '就是', '一个天', '一个', '地', '。', '33', '元', '的', '羊肉', '数量', '不', '多', '，', '但肉', '品质', '比', '原来', '要', '好', '，', '涮', '时', '不散', '，

100000 ,input_list: ['袍哥', '码头', '开', '了', '有', '段时间', '，', '听闻', '熟人', '介绍', '味道', '一级', '棒', '，', '拉上', '小伙伴', '，', '感受一下', '。', '由于', '省府', '路', '外围', '在', '城市', '改造', '，', '不太好', '通行', '，', '好', '在', '有', '指示牌', '，', '永辉', '超市', '旁边', '的', '小', '阶梯', '上去', '就是', '了', '。', '店铺', '环境', '还是', '可以', '的', '，', '一', '进门', '案', '台上', '立着', '关公', '塑像', '，', '墙上', '装饰', '着', '斗笠', '蓑衣', '，', '大厅', '还', '摆设', '着', '一张', '仿古', '实木', '书桌', '，', '上面', '摆放着', '毛笔', '，', '宣纸', '，', '墨盘', '，', '真是', '古色古香', '。', '虽然', '正值', '饭点', '，', '然而', '店里', '食客', '并', '不是', '很多', '，', '只有', '熙熙攘攘', '的', '2', '～', '3', '座', '食客', '，', '许是', '交通不便', '吧', '，', '大家', '不', '方便', '过来', '。', '点', '的', '是', '三鲜', '钙骨', '红油', '鸳鸯锅', '，', '红油', '锅底', '偏麻', '，', '辣度', '适中', '，', '但是', '感觉', '红油', '锅底', '味道', '不太', '正宗', '，', '味道', '怪怪的', '，', '没有', '涮', '这边', '。', '三鲜', '底料', '足', '，', '很', '鲜美', '，', '有', '玉米', '，', '大葱', '，', '火腿肠', '。', '看起来', '很', '诱人', '。', '雪花', '肥牛', '卷', '：', '不', '新鲜', '！', '！', '！', '涮', '的',

10000 ,input_list: ['今日', '在', '大众', '点评', '瞎逛', '，', '突然', '发现', '自家', '楼后', '的', '鲜奶', '吧', '竟然', '有', '团购', '，', '那', '还', '等', '什么', '，', '原价', '15', '，', '点评', '网', '优惠', '5', '，', '再用', '5', '元', '优惠价', '搞定', '，', '最后', '花费', '5', '元', '，', '嘎嘎', '！', '该店', '是', '个', '小', '店铺', '，', '专营', '外卖', '，', '店家', '小哥', '人', '不错', '，', '耐心', '细致', '的', '为', '我', '这个', '断奶', '30', '多年', '的', '吃货', '讲解', '双皮奶', '与', '奶昔', '，', '感慨万千', '！', '今晚', '自制', '火锅', '，', '用', '它们', '来', '做', '饮品', '及餐', '后', '甜点', '感觉', '不错', '，', '老婆', '狂赞', '中', '…', '…', '因为', '点评', '要求', '3', '张', '照片', '，', '而', '我', '去', '店', '时刚', '买', '完菜', '，', '实在', '腾', '不', '出空', '拍照', '，', '而', '这个', '套餐', '一张', '照片', '足矣', '表现', '，', '所以', '上传', '三张', '一致', '照片', '与', '大家', '分享', '，', '由此', '带来', '不便', '，', '还', '望', '谅解', '！']
10000 ,x: [4717, 17, 163, 103, 22087, 4, 1356, 244, 3049, 26614, 5, 3597, 53, 674, 14, 114, 4, 193, 20, 104, 92, 4, 1017, 732, 4, 103, 1464, 389, 256, 4, 6634, 256, 68, 11867, 4642, 4, 112, 5453

process_data.ended...
X_train: (105000,) ;Y_train: (105000, 80)
X_valid: (15000,) ;Y_valid: (15000, 80)
X_test: (15000,)
X_train[0]: [17, 1, 390, 4, 52, 9, 18, 4, 47, 13, 10163, 5, 4, 36, 490, 7, 1160, 861, 11, 31, 2804, 4, 59, 9, 1123, 6, 1067, 785, 26, 4, 483, 5, 60, 422, 159, 4, 1307, 513, 15, 2267, 1425, 141, 4, 596, 12, 1936, 21, 4, 16, 161, 10, 2127, 86, 11806, 4, 1039, 66, 937, 5, 19, 9, 1309, 4, 75, 151, 16, 20, 190, 7, 3227, 124, 6, 831, 359, 179, 34, 2257, 4, 360, 9304, 4, 314, 31, 1886, 4, 62, 22, 142, 8, 5132, 7230, 5, 582, 146, 334, 463, 115, 232, 4, 302, 37, 1330, 359, 208, 5, 4, 540, 26, 6, 36, 70, 212, 7, 2315, 548, 4, 5029, 484, 4, 929, 3571, 9, 18, 4, 2056, 7, 1729, 4180, 4, 9, 1456, 4, 129, 697, 2129, 7, 4, 1353, 11, 21, 6, 3105, 240, 2890, 4, 1230, 5573, 213, 4, 11, 64, 9, 345, 6, 47681, 5, 675, 9, 14, 1545, 4, 37, 51, 7972, 7, 6]


In [98]:
# pad input x to fixed length
import tensorflow as tf
pad_sequence = tf.contrib.keras.preprocessing.sequence.pad_sequences
print("max_sequence_length:",max_sequence_length)
X_train = pad_sequence(np.array(X_train),maxlen=max_sequence_length,padding='pre',truncating='pre',value = 0)
X_valid = pad_sequence(np.array(X_valid),maxlen=max_sequence_length,padding='pre',truncating='pre',value = 0)
X_test = pad_sequence(np.array(X_test),maxlen=max_sequence_length,padding='pre',truncating='pre',value = 0)

X_train=np.array(X_train)
Y_train=np.array(Y_train)
X_valid=np.array(X_valid)
Y_valid=np.array(Y_valid)
X_test=np.array(X_test)
print("X_train:",X_train.shape,";Y_train:",Y_train.shape)
print("X_valid:",X_valid.shape,";Y_valid:",Y_valid.shape)
print("X_test:",X_test.shape)
print("X_train[0]:",X_train[0])

max_sequence_length: 400
X_train: (105000, 400) ;Y_train: (105000, 80)
X_valid: (15000, 400) ;Y_valid: (15000, 80)
X_test: (15000, 400)
X_train[0]: [    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0 

In [99]:
 # save train/validation/test set into cache file
cache_path='./preprocess_word/train_valid_test_vocab_cache.pik'
if not os.path.exists(cache_path):
        with open(cache_path, 'ab') as data_f:
            print("going to save cache file of vocab of words and labels")
            pickle.dump((X_train,Y_train,X_valid,Y_valid,X_test,vocab_word2index,label2index), data_f)    
            print("write train/validation/test set into cache file completed.")

going to save cache file of vocab of words and labels
write train/validation/test set into cache file completed.
